In [1]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import torch
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QTextEdit, QLineEdit, QLabel
from PyQt5.QtCore import Qt, QTimer
import sys
import threading


/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Seleziona il dispositivo: GPU, MPS, o CPU
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPU NVIDIA
elif torch.backends.mps.is_available():
    device = torch.device("mps")  # GPU Apple (M1/M2)
else:
    device = torch.device("cpu")  # CPU come fallback generico

print(f"Using device: {device}")

Using device: mps


In [3]:
# Save the model (only one time we need to do this)

# Carica il modello GPT-Neo e il tokenizer
#model_name = "EleutherAI/gpt-neo-1.3B"  # Usa GPT-Neo 1.3B o la versione che preferisci
#model = GPTNeoForCausalLM.from_pretrained(model_name)
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)

#model_path = "gpt-neo-1.3B_model"

# Salva il modello e il tokenizer nella cartella personalizzata
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

In [4]:
# Carica il modello e il tokenizer
model_path = "gpt-neo-1.3B_model"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPTNeoForCausalLM.from_pretrained(model_path)

model = model.to(device)

print("Modello e tokenizer caricati con successo!")

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.21it/s]


Modello e tokenizer caricati con successo!


In [5]:
tokenizer.pad_token = tokenizer.eos_token  # Usa eos_token come pad_token

# Funzione per generare risposte con opzioni migliorate
def generate_response(prompt, max_length=75):
    # Tokenizza l'input e sposta i tensori sul dispositivo (GPU o CPU)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, return_attention_mask=True).to(device)
    
    # Usa solo 'input_ids' per la generazione
    outputs = model.generate(
        inputs['input_ids'],  # Passa solo input_ids
        attention_mask=inputs['attention_mask'],  # Passa l'attenzione mask
        max_length=max_length,
        num_return_sequences=1, 
        temperature=0.7, 
        top_p=0.5,
        top_k=50,
        repetition_penalty=5.0,
        length_penalty=5.0,
    )
    
    # Decodifica la risposta generata
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
class ChatBotApp(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Chatbot GPT-Neo")
        self.setGeometry(100, 100, 600, 700)

        # Layout verticale
        self.layout = QVBoxLayout()

        # Etichetta per il messaggio di attesa (casella fissa)
        self.wait_label = QLabel('The answer may take a few seconds ...', self)
        self.layout.addWidget(self.wait_label)

        # Finestra di chat
        self.chat_window = QTextEdit(self)
        self.chat_window.setReadOnly(True)  # La finestra di chat è in sola lettura
        self.layout.addWidget(self.chat_window)

        # Casella di input per l'utente
        self.user_input_box = QLineEdit(self)
        self.layout.addWidget(self.user_input_box)

        # Bottone per inviare il messaggio
        self.send_button = QPushButton("Send", self)
        self.send_button.clicked.connect(self.send_message)
        self.layout.addWidget(self.send_button)

        # Imposta il layout
        self.setLayout(self.layout)

    def send_message(self):
        user_input = self.user_input_box.text()
        if user_input.strip():
            # Mostra il messaggio dell'utente nella finestra di chat
            self.chat_window.append(f'<b>You:</b> {user_input}')
            
            # Pulisci la casella di input
            self.user_input_box.clear()

            # Mostra la casella di attesa
            self.wait_label.setText('La risposta potrebbe impiegarci qualche secondo...')

            # Usa threading per evitare che il kernel crashi
            threading.Thread(target=self.generate_and_display_response, args=(user_input,)).start()

    def generate_and_display_response(self, user_input):
        """Genera la risposta del bot e la visualizza"""
        # Genera la risposta dal modello GPT-Neo
        response = generate_response(user_input)

        # Rimuovi la casella di attesa e aggiungi la risposta del bot
        self.wait_label.setText('')  # Rimuove il messaggio di attesa

        # Aggiungi la risposta del bot nella chat
        self.chat_window.append(f'<b>Bot:</b> {response}')

In [7]:
app = QApplication(sys.argv)
window = ChatBotApp()
window.show()
app.exec_()

2025-01-13 20:45:58.641 Python[13969:337501] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/manuelemustari/Library/Python/3.9/lib/python/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/Users/manuelemustari/Library/Python/3.9/lib/py

0